# Validación y Diagnóstico Multi-Horizonte
# Análisis Comparativo: Mensual, Trimestral y Semestral

---

## 📊 Objetivo

Este cuaderno implementa el **diagnóstico exhaustivo y validación multi-horizonte** de los modelos predictivos, evaluando la capacidad de generalización en tres horizontes temporales:

### Horizontes de Evaluación:
1. 🗓️ **Horizonte Mensual**: 12 predicciones (Enero 2026 - Diciembre 2026)
2. 📅 **Horizonte Trimestral**: 4 predicciones (Q1-2026, Q2-2026, Q3-2026, Q4-2026)
3. 📆 **Horizonte Semestral**: 2 predicciones (S1-2026, S2-2026)

### Alcance del Análisis por Horizonte:
- **Preparación de datos**: Agregación temporal específica
- **División train-test**: Entrenamiento (2022-2025) vs Validación (2026)
- **Métricas de evaluación**: MAE, RMSE, MAPE, SMAPE, MASE
- **Diagnóstico de errores**: Análisis de residuos, autocorrelación, heterocedasticidad
- **Análisis comparativo**: Trade-off precisión vs granularidad
- **Interpretabilidad**: Importancia de variables, componentes estacionales

---

**Autor**: Sistema de Análisis Predictivo  
**Fecha**: Febrero 2026  
**Versión**: 1.0  
**Criterios de Éxito**: MAPE Mensual <12%, Trimestral <8%, Semestral <6%

## 1. Configuración del Entorno

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
from scipy import stats
from statsmodels.stats.diagnostic import acorr_ljungbox, het_breuschpagan
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Entorno configurado exitosamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Funciones Auxiliares para Métricas y Diagnóstico

In [ ]:
def calcular_metricas(y_true, y_pred, nombre_horizonte=""):
    """
    Calcula métricas estándar de evaluación para series de tiempo
    
    Parameters:
    -----------
    y_true : array-like
        Valores reales
    y_pred : array-like
        Valores predichos
    nombre_horizonte : str
        Nombre del horizonte temporal (Mensual, Trimestral, Semestral)
    
    Returns:
    --------
    dict : Diccionario con las métricas calculadas
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # MAE (Mean Absolute Error)
    mae = mean_absolute_error(y_true, y_pred)
    
    # RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # MAPE (Mean Absolute Percentage Error)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    # SMAPE (Symmetric Mean Absolute Percentage Error)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100
    
    # MASE (Mean Absolute Scaled Error) - usando naive forecast
    naive_forecast_error = np.mean(np.abs(np.diff(y_true)))
    mase = mae / naive_forecast_error if naive_forecast_error > 0 else np.inf
    
    # Dirección correcta (predicción de tendencia)
    if len(y_true) > 1:
        direccion_real = np.sign(np.diff(y_true))
        direccion_pred = np.sign(np.diff(y_pred))
        direccion_correcta = np.mean(direccion_real == direccion_pred) * 100
    else:
        direccion_correcta = np.nan
    
    # R² Score
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
    
    metricas = {
        'Horizonte': nombre_horizonte,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE (%)': mape,
        'SMAPE (%)': smape,
        'MASE': mase,
        'R²': r2,
        'Dirección_Correcta (%)': direccion_correcta
    }
    
    return metricas


def diagnostico_residuos(residuos, nombre_horizonte=""):
    """
    Realiza diagnóstico completo de residuos
    
    Parameters:
    -----------
    residuos : array-like
        Residuos del modelo (y_true - y_pred)
    nombre_horizonte : str
        Nombre del horizonte temporal
    
    Returns:
    --------
    dict : Diccionario con resultados de tests estadísticos
    """
    residuos = np.array(residuos)
    
    # Test de normalidad (Shapiro-Wilk)
    if len(residuos) >= 3:
        shapiro_stat, shapiro_p = stats.shapiro(residuos)
    else:
        shapiro_stat, shapiro_p = np.nan, np.nan
    
    # Test de Jarque-Bera
    if len(residuos) >= 3:
        jb_stat, jb_p = stats.jarque_bera(residuos)
    else:
        jb_stat, jb_p = np.nan, np.nan
    
    # Test de autocorrelación (Ljung-Box)
    if len(residuos) >= 3:
        try:
            lb_test = acorr_ljungbox(residuos, lags=min(10, len(residuos)//2), return_df=True)
            lb_p = lb_test['lb_pvalue'].iloc[0]
        except:
            lb_p = np.nan
    else:
        lb_p = np.nan
    
    diagnostico = {
        'Horizonte': nombre_horizonte,
        'Media_Residuos': np.mean(residuos),
        'Std_Residuos': np.std(residuos),
        'Shapiro_p_value': shapiro_p,
        'JarqueBera_p_value': jb_p,
        'LjungBox_p_value': lb_p,
        'Normalidad': 'Pasa' if shapiro_p > 0.05 else 'Falla' if not np.isnan(shapiro_p) else 'N/A',
        'Autocorrelacion': 'No detectada' if lb_p > 0.05 else 'Detectada' if not np.isnan(lb_p) else 'N/A'
    }
    
    return diagnostico


def evaluar_criterio_exito(metricas, horizonte):
    """
    Evalúa si el modelo cumple los criterios de éxito definidos
    
    Criterios:
    - Mensual: MAPE < 12%
    - Trimestral: MAPE < 8%
    - Semestral: MAPE < 6%
    """
    criterios = {
        'Mensual': 12,
        'Trimestral': 8,
        'Semestral': 6
    }
    
    umbral = criterios.get(horizonte, 15)
    mape = metricas['MAPE (%)']
    
    if mape < umbral:
        return f"✅ APROBADO (MAPE {mape:.2f}% < {umbral}%)"
    else:
        return f"❌ NO APROBADO (MAPE {mape:.2f}% >= {umbral}%)"

print("✅ Funciones auxiliares cargadas")

## 3. Carga de Datos Procesados

In [ ]:
# Cargar datos con features engineering ya aplicado
print("📁 Cargando datos procesados...")

# TODO: Ajustar según output del cuaderno 02_Feature_Engineering
try:
    df_features = pd.read_csv('../data/features/datos_con_features.csv')
    print(f"✅ Datos cargados: {df_features.shape[0]:,} registros")
    print(f"   Columnas disponibles: {df_features.shape[1]}")
except FileNotFoundError:
    print("⚠️  Archivo no encontrado. Ejecutar primero 02_Feature_Engineering.ipynb")
    df_features = None

---
# SEGMENTO 1: HORIZONTE MENSUAL
---

## 📅 Diagnóstico del Horizonte Mensual

**Configuración:**
- **Entrenamiento**: Enero 2022 - Diciembre 2025 (48 meses)
- **Validación**: Enero 2026 - Diciembre 2026 (12 meses)
- **Criterio de éxito**: MAPE < 12%, Dirección correcta >75%
- **Objetivo**: Capturar dinámicas de corto plazo y estacionalidad mensual

### 3.1. Preparación de Datos Mensuales

In [ ]:
# TODO: Ajustar según estructura real
# Aquí va la lógica para:
# 1. Agregar datos por mes
# 2. Dividir en train (2022-2025) y test (2026)
# 3. Preparar features específicos para horizonte mensual

print("⏳ Preparando agregación mensual...")
print("\nPendiente: Implementar después de Feature Engineering")
print("\nEstructura esperada:")
print("  - df_mensual: DataFrame con recaudo agregado por mes")
print("     - train_mensual: 2022-2025 (48 meses)")
print("     - test_mensual: 2026 (12 meses)")

### 3.2. Análisis Exploratorio del Horizonte Mensual

In [ ]:
# Placeholder para visualización de serie temporal mensual
print("📊 Visualizaciones a generar:")
print("  1. Serie temporal completa (2022-2025 + 2026)")
print("  2. Descomposición: Tendencia, Estacionalidad, Residuos")
print("  3. Análisis de autocorrelación (ACF/PACF)")
print("  4. Distribución de valores por mes")
print("  5. Heatmap de estacionalidad mensual")

### 3.3. Modelado y Predicción Mensual

**Nota**: Los modelos se entrenan en el cuaderno `03_Modelos_Predictivos.ipynb`.  
Aquí solo cargamos las predicciones ya generadas y realizamos la evaluación.

In [ ]:
# TODO: Cargar predicciones mensuales de cada modelo
# Ejemplo:
# predicciones_mensual = {
#     'Prophet': pred_prophet_mensual,
#     'NeuralProphet': pred_neuralprophet_mensual,
#     'NHITS': pred_nhits_mensual,
#     'XGBoost': pred_xgboost_mensual,
#     'Ensemble': pred_ensemble_mensual
# }

print("⏳ Pendiente: Cargar predicciones de modelos entrenados")

### 3.4. Evaluación de Métricas - Horizonte Mensual

In [ ]:
# Placeholder para cálculo de métricas
print("📊 EVALUACIÓN DE MÉTRICAS - HORIZONTE MENSUAL")
print("="*80)

# Ejemplo de estructura de resultados
resultados_mensual = pd.DataFrame({
    'Modelo': ['Prophet', 'NeuralProphet', 'NHITS', 'XGBoost', 'Ensemble'],
    'MAE': [np.nan] * 5,
    'RMSE': [np.nan] * 5,
    'MAPE (%)': [np.nan] * 5,
    'SMAPE (%)': [np.nan] * 5,
    'R²': [np.nan] * 5,
    'Dirección_Correcta (%)': [np.nan] * 5
})

print("\n⏳ Pendiente: Calcular métricas una vez se tengan predicciones")
print("\nEstructura de tabla de resultados:")
display(resultados_mensual)

### 3.5. Diagnóstico de Residuos - Horizonte Mensual

In [ ]:
print("🔍 DIAGNÓSTICO DE RESIDUOS - HORIZONTE MENSUAL")
print("="*80)
print("\nTests a realizar:")
print("  1. Test de Normalidad (Shapiro-Wilk)")
print("  2. Test de Jarque-Bera")
print("  3. Test de Autocorrelación (Ljung-Box)")
print("  4. Gráficos Q-Q")
print("  5. Histograma de residuos")
print("  6. Gráfico de residuos vs tiempo")

### 3.6. Visualización Comparativa - Predicciones vs Reales (Mensual)

In [ ]:
# Placeholder para gráfico interactivo
print("📈 Gráfico a generar: Predicciones vs Valores Reales (2026)")
print("   - Línea de valores reales")
print("   - Líneas de predicciones de cada modelo")
print("   - Intervalos de confianza")
print("   - Marcadores de eventos especiales (enero, marzo)")

### 3.7. Conclusiones del Horizonte Mensual

In [ ]:
print("="*80)
print("RESUMEN EJECUTIVO - HORIZONTE MENSUAL")
print("="*80)
print("\n📊 MÉTRICAS CONSOLIDADAS:")
print("  Mejor modelo: [Pendiente]")
print("  MAPE del mejor modelo: [Pendiente]%")
print("  Criterio de éxito: MAPE < 12%")
print("  Resultado: [Pendiente]")
print("\n✅ FORTALEZAS:")
print("  [Pendiente análisis]")
print("\n⚠️  DEBILIDADES:")
print("  [Pendiente análisis]")
print("\n💡 RECOMENDACIONES:")
print("  [Pendiente análisis]")

---
# SEGMENTO 2: HORIZONTE TRIMESTRAL
---

## 📊 Diagnóstico del Horizonte Trimestral

**Configuración:**
- **Entrenamiento**: Q1-2022 a Q4-2025 (16 trimestres)
- **Validación**: Q1-2026, Q2-2026, Q3-2026, Q4-2026 (4 trimestres)
- **Criterio de éxito**: MAPE < 8%, Dirección correcta >85%
- **Objetivo**: Suavizar ruido mensual y capturar tendencias estructurales

### 4.1. Preparación de Datos Trimestrales

In [ ]:
print("⏳ Preparando agregación trimestral...")
print("\nPendiente: Implementar después de Feature Engineering")
print("\nEstructura esperada:")
print("  - df_trimestral: DataFrame con recaudo agregado por trimestre")
print("     - train_trimestral: Q1-2022 a Q4-2025 (16 trimestres)")
print("     - test_trimestral: Q1-2026 a Q4-2026 (4 trimestres)")
print("\nAgregación:")
print("  - Q1: Enero + Febrero + Marzo")
print("  - Q2: Abril + Mayo + Junio")
print("  - Q3: Julio + Agosto + Septiembre")
print("  - Q4: Octubre + Noviembre + Diciembre")

### 4.2. Análisis Exploratorio del Horizonte Trimestral

In [ ]:
print("📊 Visualizaciones a generar:")
print("  1. Serie temporal trimestral (2022-2025 + 2026)")
print("  2. Descomposición trimestral: Tendencia, Estacionalidad, Residuos")
print("  3. Análisis de autocorrelación (ACF/PACF) trimestral")
print("  4. Box plots por trimestre (Q1, Q2, Q3, Q4)")
print("  5. Heatmap de estacionalidad trimestral")
print("  6. Comparación de varianza: mensual vs trimestral")

### 4.3. Evaluación de Métricas - Horizonte Trimestral

In [ ]:
print("📊 EVALUACIÓN DE MÉTRICAS - HORIZONTE TRIMESTRAL")
print("="*80)

# Ejemplo de estructura de resultados
resultados_trimestral = pd.DataFrame({
    'Modelo': ['Prophet', 'NeuralProphet', 'NHITS', 'XGBoost', 'Ensemble'],
    'MAE': [np.nan] * 5,
    'RMSE': [np.nan] * 5,
    'MAPE (%)': [np.nan] * 5,
    'SMAPE (%)': [np.nan] * 5,
    'R²': [np.nan] * 5,
    'Dirección_Correcta (%)': [np.nan] * 5
})

print("\n⏳ Pendiente: Calcular métricas una vez se tengan predicciones")
display(resultados_trimestral)

### 4.4. Diagnóstico de Residuos - Horizonte Trimestral

In [ ]:
print("🔍 DIAGNÓSTICO DE RESIDUOS - HORIZONTE TRIMESTRAL")
print("="*80)
print("\nMismos tests que horizonte mensual, adaptados a frecuencia trimestral")

### 4.5. Conclusiones del Horizonte Trimestral

In [ ]:
print("="*80)
print("RESUMEN EJECUTIVO - HORIZONTE TRIMESTRAL")
print("="*80)
print("\n📊 MÉTRICAS CONSOLIDADAS:")
print("  Mejor modelo: [Pendiente]")
print("  MAPE del mejor modelo: [Pendiente]%")
print("  Criterio de éxito: MAPE < 8%")
print("  Resultado: [Pendiente]")
print("\n✅ VENTAJAS vs Horizonte Mensual:")
print("  [Pendiente análisis comparativo]")
print("\n💡 RECOMENDACIONES:")
print("  [Pendiente análisis]")

---
# SEGMENTO 3: HORIZONTE SEMESTRAL
---

## 📆 Diagnóstico del Horizonte Semestral

**Configuración:**
- **Entrenamiento**: S1-2022 a S2-2025 (8 semestres)
- **Validación**: S1-2026, S2-2026 (2 semestres)
- **Criterio de éxito**: MAPE < 6%, Robustez ante variabilidad extrema
- **Objetivo**: Identificar tendencias de largo plazo y ciclos anuales

### 5.1. Preparación de Datos Semestrales

In [ ]:
print("⏳ Preparando agregación semestral...")
print("\nPendiente: Implementar después de Feature Engineering")
print("\nEstructura esperada:")
print("  - df_semestral: DataFrame con recaudo agregado por semestre")
print("     - train_semestral: S1-2022 a S2-2025 (8 semestres)")
print("     - test_semestral: S1-2026, S2-2026 (2 semestres)")
print("\nAgregación:")
print("  - S1: Enero + Febrero + Marzo + Abril + Mayo + Junio")
print("  - S2: Julio + Agosto + Septiembre + Octubre + Noviembre + Diciembre")

### 5.2. Análisis Exploratorio del Horizonte Semestral

In [ ]:
print("📊 Visualizaciones a generar:")
print("  1. Serie temporal semestral (2022-2025 + 2026)")
print("  2. Tendencia de largo plazo")
print("  3. Análisis de ciclo anual (S1 vs S2)")
print("  4. Box plots por semestre")
print("  5. Comparación de varianza: mensual vs trimestral vs semestral")
print("  6. Crecimiento anual acumulado")

### 5.3. Evaluación de Métricas - Horizonte Semestral

In [ ]:
print("📊 EVALUACIÓN DE MÉTRICAS - HORIZONTE SEMESTRAL")
print("="*80)

resultados_semestral = pd.DataFrame({
    'Modelo': ['Prophet', 'NeuralProphet', 'NHITS', 'XGBoost', 'Ensemble'],
    'MAE': [np.nan] * 5,
    'RMSE': [np.nan] * 5,
    'MAPE (%)': [np.nan] * 5,
    'SMAPE (%)': [np.nan] * 5,
    'R²': [np.nan] * 5
})

print("\n⏳ Pendiente: Calcular métricas una vez se tengan predicciones")
print("\nNota: Con solo 2 predicciones (S1-2026, S2-2026),")
print("      la métrica de dirección correcta es menos informativa")
display(resultados_semestral)

### 5.4. Conclusiones del Horizonte Semestral

In [ ]:
print("="*80)
print("RESUMEN EJECUTIVO - HORIZONTE SEMESTRAL")
print("="*80)
print("\n📊 MÉTRICAS CONSOLIDADAS:")
print("  Mejor modelo: [Pendiente]")
print("  MAPE del mejor modelo: [Pendiente]%")
print("  Criterio de éxito: MAPE < 6%")
print("  Resultado: [Pendiente]")
print("\n✅ VENTAJAS para Planificación Estratégica:")
print("  [Pendiente análisis]")
print("\n💡 RECOMENDACIONES:")
print("  [Pendiente análisis]")

---
# ANÁLISIS COMPARATIVO MULTI-HORIZONTE
---

## 🎯 Comparación Directa entre los Tres Horizontes

### 6.1. Tabla Consolidada de Métricas por Horizonte

In [ ]:
print("📊 TABLA COMPARATIVA DE MÉTRICAS POR HORIZONTE")
print("="*80)

# Estructura de tabla consolidada
comparacion_horizontes = pd.DataFrame({
    'Horizonte': ['Mensual', 'Trimestral', 'Semestral'],
    'N_Predicciones': [12, 4, 2],
    'Mejor_Modelo': ['Pendiente', 'Pendiente', 'Pendiente'],
    'MAE': [np.nan, np.nan, np.nan],
    'RMSE': [np.nan, np.nan, np.nan],
    'MAPE (%)': [np.nan, np.nan, np.nan],
    'Criterio_Exito': ['<12%', '<8%', '<6%'],
    'Resultado': ['Pendiente', 'Pendiente', 'Pendiente']
})

display(comparacion_horizontes)

print("\n🎯 ANÁLISIS ESPERADO:")
print("  - MAPE debe decrecer al aumentar horizonte (semestral < trimestral < mensual)")
print("  - Estabilidad aumenta con horizonte más largo")
print("  - Granularidad disminuye con horizonte más largo")

### 6.2. Visualización Comparativa: Trade-off Precisión vs Granularidad

In [ ]:
print("📈 Gráficos a generar:")
print("  1. Gráfico de barras: MAPE por horizonte y por modelo")
print("  2. Gráfico de líneas: Convergencia de errores al aumentar horizonte")
print("  3. Heatmap: Errores absolutos por periodo y horizonte")
print("  4. Radar chart: Métricas normalizadas por horizonte")
print("  5. Dashboard interactivo con los 3 horizontes simultáneamente")

### 6.3. Matriz de Decisión: Selección del Horizonte Óptimo

In [ ]:
print("🎯 MATRIZ DE DECISIÓN")
print("="*80)

print("\nCRITERIOS DE SELECCIÓN:")
print("\n1️⃣  Si MAPE_mensual < 12%:")
print("     → USO OPERATIVO MENSUAL")
print("     → Ventaja: Máxima granularidad para decisiones tácticas")

print("\n2️⃣  Si MAPE_mensual >= 12% pero MAPE_trimestral < 8%:")
print("     → USO TRIMESTRAL")
print("     → Ventaja: Balance entre precisión y detalle")

print("\n3️⃣  Si ambos fallan:")
print("     → USO SEMESTRAL (baseline estratégico)")
print("     → Ventaja: Tendencias de largo plazo confiables")

print("\n4️⃣  RECOMENDACIÓN FINAL:")
print("     → ENSEMBLE MULTI-HORIZONTE")
print("        • Horizonte mensual para proyecciones de corto plazo (0-3 meses)")
print("        • Horizonte trimestral para planificación táctica (3-6 meses)")
print("        • Horizonte semestral para planificación estratégica (6-12 meses)")
print("        • Combinar predicciones ponderadas por precisión histórica")

### 6.4. Dashboard Ejecutivo Multi-Horizonte

In [ ]:
print("📊 COMPONENTES DEL DASHBOARD EJECUTIVO:")
print("="*80)

print("\n1. PANEL SUPERIOR: Métricas Clave")
print("   - KPI cards con MAPE de cada horizonte")
print("   - Semáforo de aprobación (verde/amarillo/rojo)")
print("   - Recomendación principal")

print("\n2. PANEL CENTRAL: Visualización Temporal")
print("   - Serie temporal con 3 capas (mensual/trimestral/semestral)")
print("   - Marcadores de eventos especiales")
print("   - Intervalos de confianza")

print("\n3. PANEL INFERIOR: Análisis de Errores")
print("   - Distribución de residuos por horizonte")
print("   - Heatmap de errores temporales")
print("   - Tabla de métricas detalladas")

print("\n⏳ Pendiente: Implementar una vez se tengan todos los resultados")

---
# CONCLUSIONES FINALES Y RECOMENDACIONES
---

In [ ]:
print("="*80)
print("INFORME EJECUTIVO FINAL - VALIDACIÓN MULTI-HORIZONTE")
print("="*80)

print("\n📊 RESUMEN DE RESULTADOS:")
print("   [Pendiente: Completar después de tener predicciones]")

print("\n🎯 HORIZONTE RECOMENDADO:")
print("   [Pendiente: Determinar según métricas]")

print("\n✅ FORTALEZAS DEL MODELO SELECCIONADO:")
print("   [Pendiente: Análisis de ventajas]")

print("\n⚠️  LIMITACIONES Y RIESGOS:")
print("   [Pendiente: Identificar puntos débiles]")

print("\n💡 RECOMENDACIONES OPERATIVAS:")
print("   1. [Pendiente]")
print("   2. [Pendiente]")
print("   3. [Pendiente]")

print("\n📅 PRÓXIMOS PASOS:")
print("   1. Implementar modelo en producción")
print("   2. Configurar monitoreo de drift")
print("   3. Establecer protocolo de reentrenamiento")
print("   4. Documentar en informe Word para entrega (4 de marzo)")

print("\n" + "="*80)
print("FIN DEL ANÁLISIS")
print("="*80)